<a href="https://colab.research.google.com/github/jingxuchen19/ECON5200-Applied-Data-Analytics-in-Economics/blob/main/Class%206/Lab_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import seaborn as sns
import pandas as pd
import numpy as np

df = sns.load_dataset('titanic')
print(f"Toal Population: {len(df)}")
print(f"Population Survival Rate: {df['survived'].mean():.4f}")

np.random.seed(2026)
indices = np.random.permutation(df.index)

Toal Population: 891
Population Survival Rate: 0.3838


In [13]:
from numpy.matrixlib import test
split_point = int(len(df.index) * 0.8)

train_idx = indices[:split_point]
test_idx = indices[split_point:]

train_set = df.loc[train_idx]
test_set = df.loc[test_idx]

train_surv = train_set['survived'].mean()
test_surv = test_set['survived'].mean()
delta = abs(train_surv - test_surv)

print(f"Train Survival Rate: {train_surv:.4f}")
print(f"Test Survival Rate: {test_surv:.4f}")
print(f"Sampling Bias (Delta):{delta:.4f}")

Train Survival Rate: 0.3736
Test Survival Rate: 0.4246
Sampling Bias (Delta):0.0510


In [14]:
df.corr(numeric_only=True)

,survived,pclass,age,sibsp,parch,fare,adult_male,alone
survived,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307,-0.557080,-0.203367
pclass,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500,0.094035,0.135207
age,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067,0.280328,0.198270
sibsp,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651,-0.253586,-0.584471
parch,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225,-0.349943,-0.583398
fare,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000,-0.182024,-0.271832
adult_male,-0.557080,0.094035,0.280328,-0.253586,-0.349943,-0.182024,1.000000,0.404744
alone,-0.203367,0.135207,0.198270,-0.584471,-0.583398,-0.271832,0.404744,1.000000


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df,test_size=0.2,stratify=df['pclass'],random_state=2026)

print("\n--- Stratified Split ---")
print(f"Train Class Dist:\n", X_train['pclass'].value_counts(normalize=True))
print(f"Test Class Dist:\n", X_test['pclass'].value_counts(normalize=True))


--- Stratified Split ---
Train Class Dist:
 pclass
3    0.550562
1    0.242978
2    0.206461
Name: proportion, dtype: float64
Test Class Dist:
 pclass
3    0.553073
1    0.240223
2    0.206704
Name: proportion, dtype: float64


In [20]:
population_probs = df['pclass'].value_counts(normalize=True).sort_index()

sample_size = 100
bad_sample = df.sample(n=sample_size, random_state=2026)

observed = (
    bad_sample['pclass']
    .value_counts()
    .sort_index()
    .values
)

expected = (population_probs * sample_size).values

print(f"Observed: {observed}")
print(f"Expected: {expected}")

from scipy.stats import chisquare
chi2_stat, p_value = chisquare(f_obs=observed, f_exp=expected)

print("-" * 40)
print(f"Chi-Square Statistic: {chi2_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

if p_value < 0.01:
    print("CRITICAL FAILURE: Sample Ratio Mismatch (SRM) Detected.")
else:
    print("PASS: Variance is within natural limits.")

Observed: [21 22 57]
Expected: [24.24242424 20.65095398 55.10662177]
----------------------------------------
Chi-Square Statistic: 0.5869
P-Value: 0.7457
PASS: Variance is within natural limits.
